In [1]:
# === CONFIGURATION ===
MODEL_NAME = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
JSON_PATH = "./classified_Empathy_Supervisor.json"  # 원본 JSON (CBT 감정공감 데이터)
JSONL_PATH = "./cbt_empathy_train.jsonl"  # 변환 후 저장 경로
OUTPUT_DIR = "./empathy_adapter"

In [2]:
import json

with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

with open(JSONL_PATH, "w", encoding="utf-8") as f_out:
    for item in data:
        text = item["content"].replace("{이름}", "당신")
        entry = {
            "instruction": "당신은 감정 공감을 잘하는 상담사입니다. 적절한 공감 표현을 작성하세요.",
            "input": "",
            "output": text
        }
        f_out.write(json.dumps(entry, ensure_ascii=False) + "\n")

In [3]:
!pip install -q peft transformers datasets accelerate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.5.6 requires torchvision, which is not installed.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.7.0 which is incompatible.
unsloth 2025.5.6 requires transformers!=4.47.0,==4.51.3, but you have transformers 4.52.0 which is incompatible.
unsloth-zoo 2025.5.7 requires transformers!=4.47.0,==4.51.3, but you have transformers 4.52.0 which is incompatible.


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, device_map="auto")

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # llama-3 계열 기준
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
from datasets import load_dataset

dataset = load_dataset("json", data_files=JSONL_PATH)["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
pip show trl

Name: trl
Version: 0.15.2
Summary: Train transformer language models with reinforcement learning.
Home-page: https://github.com/huggingface/trl
Author: Leandro von Werra
Author-email: leandro.vonwerra@gmail.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: accelerate, datasets, rich, transformers
Required-by: unsloth, unsloth_zoo
Note: you may need to restart the kernel to use updated packages.


In [16]:
def format_example(e):
    prompt = f"Instruct: {e['instruction']}\nInput: {e['input']}\nOutput: {e['output']}"
    tokens = tokenizer(prompt, max_length=512, truncation=True)
    return tokenizer.decode(tokens["input_ids"], skip_special_tokens=True)

In [17]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    formatting_func=format_example,
    tokenizer=tokenizer
)

/tmp/ipykernel_10440/874588028.py:17: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Applying formatting function to train dataset:   0%|          | 0/1918 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/1918 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1918 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1918 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1918 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [14]:
import torch
torch.cuda.empty_cache()

In [18]:
trainer.train()

Step,Training Loss
10,1.718100
20,1.585800
30,1.471600
40,1.370100
50,1.288100
60,1.352900
70,1.385100
80,1.285600
90,1.386800
100,1.327400


TrainOutput(global_step=720, training_loss=1.2044493211640253, metrics={'train_runtime': 790.8254, 'train_samples_per_second': 7.276, 'train_steps_per_second': 0.91, 'total_flos': 1.91642679853056e+16, 'train_loss': 1.2044493211640253})

In [19]:
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

('./empathy_adapter/tokenizer_config.json',
 './empathy_adapter/special_tokens_map.json',
 './empathy_adapter/chat_template.jinja',
 './empathy_adapter/tokenizer.json')